# Соревнование на kaggle

По набору примеров многомерных признаков trainX и правильных ответов trainY из обучающего множества обучить модель регрессии, которую затем необходимо будет применить к набору примеров признаков из тестового множества testX и прислать файл с полученными ответами.
Сами признаки никакого осмысленного значения не несут.

In [ ]:
#Первая реализация. Результат ровный baseline
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

train_X = pd.read_csv("trainX.csv", index_col = 'Id').to_numpy()
train_Y = pd.read_csv("trainY.csv", index_col = 'Id').to_numpy()
test_X = pd.read_csv("testX.csv", index_col = 'Id').to_numpy()

model = LinearRegression()
model.fit(train_X, train_Y)

y = model.predict(test_X)
y = y.reshape(2000,)
Ypd = pd.DataFrame({'Value': y, 'Id': range(len(y))})
# Ypd.to_csv('soin_roman.csv', index=False)


In [7]:
# Вторая реализация 
import pandas as pd
import numpy as np

train_X = pd.read_csv("trainX.csv", index_col = 'Id').to_numpy()
train_Y = pd.read_csv("trainY.csv", index_col = 'Id').to_numpy()
test_X = pd.read_csv("testX.csv", index_col = 'Id').to_numpy()

class Linear_regression(object):
    
    def __init__(self, sgd, n_epoch, batch_size):
        self.sgd=sgd
        self.n_epoch=n_epoch
        self.batch_size=batch_size
        self.W = None
        self.trainX = None
        self.trainY = None
        
    def fit(self, X, y):

        self.trainX = np.array(X)
        self.trainY = np.array(y)
        self.W = np.random.rand(self.trainX.shape[1])
        start = 0
        end = self.batch_size
        for i in range(self.n_epoch):
            while end < self.trainX.shape[0]:
                self.W = self.sgd.step(self.trainX[start:end], self.trainY, self.W)
                start += self.batch_size
                end += self.batch_size
            np.random.shuffle(self.trainX)
            start = 0
            end = self.batch_size
                          
    def predict(self, X):
        
        X = np.array(X)
        n = X.shape[0]
        y = np.array([ self.W @ X[i] for i in range(n)])
        return y

class SGD(object):
    
    def __init__(self, grad, alpha):
        self.grad=grad
        self.alpha=alpha 
     
    def step(self, X, y, W):
        
        trainX=np.array(X)
        k=trainX.shape[0]
        new_weights = np.copy(W)
        S=0
        for i in range(k):
            S=S+self.grad.grad(X[i],y[i],W)   
        new_weights=new_weights-self.alpha/k*S
        return  new_weights
       
class Grad(object):
   
    def __init__(self, loss, delta):
        self.loss = loss
        self.delta = delta
        
    def grad(self, X, y, W):
        
        n_weights=W.size
        g=np.zeros(n_weights,dtype='float64')
        lost = self.loss.val
        for i in range(n_weights):
            increment=np.zeros(n_weights,dtype='float64')
            increment[i]=1 
            g[i]=(lost(X,y,W+increment*self.delta) -lost(X,y,W))/self.delta     
        return g

class Loss(object):
    
    def __init__(self, l1_coef):
        self.l1_coef=l1_coef
            
    def val(self, X, y, W):
       
        L1 = (W @ X - y)**2
        lost=0
        for i in W:
            lost += np.abs(i)
        lost=lost*self.l1_coef 
        lost=lost+L1
        return lost
        
def main(model, train_X, train_Y, test_X):
    model.fit(train_X, train_Y)
    y_pred = model.predict(test_X)
    y_pred = y_pred.reshape(2000,)
    Ypd = pd.DataFrame({'Value': y_pred, 'Id': range(len(y_pred))})
    Ypd.to_csv('soin_roman.csv', index=False)
        
#Подбор коэффицентов        
alpha = 0.0431
delta = 0.000000001
l1_coef = 0.1757
n_epoch = 2
batch_size = 9

loss = Loss(l1_coef)
grad = Grad(loss, delta)
sgd = SGD(grad, alpha)

model = Linear_regression(sgd, n_epoch, batch_size)
main(model, train_X, train_Y, test_X)

# Журнал изменений параметров и точность при них
# При вервой реализации точность была ровно baseline

# При второй реализации и таких параметрах:
# alpha = 0.0331
# delta = 0.000000001
# l1_coef = 0.1747
# n_epoch = 2
# batch_size = 8
# Точность стала 0.61261

# Потом на этих же параметрах 0.56036
# Точность снизилась. Наверное сгенерированный testY был хуже чем в прошлый раз.

#Потом при таких параметрах : 
# alpha = 0.0431
# delta = 0.000000001
# l1_coef = 0.1747
# n_epoch = 2
# batch_size = 8
# Точность стала 0.64182 .Она повысилась при увелечении шага градиентного спуска.
 
#Потом при таких параметрах : 
# alpha = 0.0431
# delta = 0.000000001
# l1_coef = 0.1747
# n_epoch = 2
# batch_size = 9
# Точность стала 0.75840 .Теперь она повысилась при увелечении батча.    

#Потом при таких параметрах : 
# alpha = 0.0431
# delta = 0.000000001
# l1_coef = 0.1757
# n_epoch = 2
# batch_size = 9
# Точность стала 0.76670 .Теперь она повысилась при увелечении коэффицента регуляризации.   

# Дальше при этих же параметрах точность понизилась и стала 0.69586  
# Потом снова на этих параметрах повысилась и стала 0.73779
